# Classificando a base de [wine](https://archive.ics.uci.edu/ml/datasets/Wine) usando um KNN
Esse notebook faz parte do material de apoio do tutorial [Introdução ao Scikit-learn - Parte 2: iniciando um projeto](http://computacaointeligente.com.br/outros/intro-sklearn-part-2/)

In [ ]:
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

## Carregando os dados 

In [ ]:
wine_dataset = load_wine()
X = wine_dataset['data']
y = wine_dataset['target']
nome_das_classes = wine_dataset.target_names
descricao = wine_dataset['DESCR']
print(descricao)

## Pre-processamento
- Dividir o dataset em treino e teste
- Normalizar os dados usando `MinMaxScaler`

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=32)
print(f"Tamanho do conjunto de treino: {len(X_treino)}")
print(f"Tamanho do conjunto de teste: {len(X_teste)}")

### Normalizando os dados

In [ ]:
normalizador = MinMaxScaler()
normalizador.fit(X_treino)
X_treino_norm = normalizador.transform(X_treino)
X_teste_norm = normalizador.transform(X_teste)

## Configurando e treinando o KNN
- Número de vizinhos = 3

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_treino_norm, y_treino)
print(f"Acurácia de treinamento: {knn.score(X_treino_norm, y_treino)}")

### Explorando alguns métodos da classe `KNeighborsClassifier`

In [ ]:
y_pred = knn.predict(X_teste_norm) # retorna a classe diretamente
y_pred_prob = knn.predict_proba(X_teste_norm) # retorna a probabilidade de cada classe
acc_teste = knn.score(X_teste_norm, y_teste)
print(f"Acurácia de teste: {acc_teste}")
print("Predições para cada amostra:")
k = 1
for l, p in zip(y_pred, y_pred_prob):
    print(f"- Amostra {k}: label = {l} | probabilidades = {p}")
    k+=1

### Relatório de classificação
- Obtendo um relatório de classificação com informações de recall, precision, F1-score.
- Imprimir a matriz de confusão

In [ ]:
relatorio = classification_report(y_teste, y_pred, target_names=nome_das_classes)
print("Relatório de classificação:")
print(relatorio)

In [ ]:
mat_conf = confusion_matrix(y_teste, y_pred)
print("Matriz de confusão:")
print(mat_conf)

# Empacotando o nosso modelo em um `pipeline`
Para fins de aprendizagem, vamos juntar todo o processo de pre-processamento e modelagem dentro de um pipeline. A principal utilidade é simplificar o código e também utilizar-lo para o busca de parâmetros. Como já foi discutido cada passo nas células anteriores, aqui vamos fazer tudo na mesma célula.

In [ ]:
knn_pipeline = Pipeline(steps=[
  ("normalizacao", MinMaxScaler()),  
  ("KNN", KNeighborsClassifier(n_neighbors=3))
])
knn_pipeline.fit(X_treino, y_treino)
y_pred = knn_pipeline.predict(X_teste)
y_pred_prob = knn_pipeline.predict_proba(X_teste)
print(f"Acurácia de treinamento: {knn_pipeline.score(X_treino, y_treino)}")

# Buscando números de vizinhos utilizando `GridSearchCV`

In [ ]:
param_busca={
  'KNN__n_neighbors': [3, 5, 7]
}
buscador = GridSearchCV(knn_pipeline, param_grid=param_busca)
buscador.fit(X, y)
print("Melhor K:", buscador.best_params_)

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(buscador.cv_results_)

# Salvando o modelo
Na documentação oficial é descrito duas maneiras de persistir modelos da `sklearn`: utilizando `pickle` ou `joblib`. Porém, a mesma recomenda a `joblib` pois ela é mais eficiente para carregar arrays com muitos dados, que é o caso da maioria dos modelos.

Vamos iniciar salvando o nosso modelo empacotado dentro do `pipeline`

In [ ]:
import joblib
joblib.dump(knn_pipeline, 'knn_pipeline.joblib') 

Para carregar, é igualmente fácil

In [ ]:
knn_pipeline_carregado = joblib.load('knn_pipeline.joblib') 
y_pred_prob = knn_pipeline_carregado.predict_proba(X_teste)
print(f"Acurácia de treinamento: {knn_pipeline_carregado.score(X_treino, y_treino)}")